![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Finteresting-problems&branch=main&subPath=notebooks/pat-results-visualizations.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Alberta Provincial Testing Results Visualization

Students in Alberta write [Diploma Exams](https://www.alberta.ca/diploma-exams.aspx) and [Provincial Achievement Tests (PATs)](https://www.alberta.ca/provincial-achievement-tests.aspx). This notebook will not include interpretation of results, please refer to the [Guidelines for Interpreting Achievement Test Multiyear Reports](https://www.alberta.ca/assets/documents/guidelines-internet-pat-multiyear-2019.pdf).

We will download the most recent PAT results data and create some visualizations. It is a fairly large dataset to download, so this first cell may take a minute or two to `►Run`.

In [ ]:
pat_url = 'https://www.alberta.ca/assets/documents/ed-pat-multiyear-sch-list.xlsx'

import requests
import pandas as pd
import plotly.express as px
r = requests.get(pat_url)
data = pd.read_excel(r.content) #, engine='openpyxl'
data

## Exploring the Data

Now that we have it stored in our DataFrame called `data`, let's have a look at some of the columns. Notice that the term [authority](https://en.wikipedia.org/wiki/List_of_school_authorities_in_Alberta) is used to refer to groups of schools.

Which authority types are represented?

In [ ]:
for authority_type in data['Authority Type'].unique():
    print(authority_type)

## Authorities

Let's list the names of the *public* authorities. Of course you can change `authority_type = 'Public'` to `authority_type = 'Separate'` or any of the types listed above.

In [ ]:
authority_type = 'Public'
for a in data[data['Authority Type']==authority_type]['Authority Name'].unique():
    print(a)

## Schools

Next we can pick a particular authority and list the schools for which we have data.

In [ ]:
authority = 'The Elk Island School Division'
for school in data[data['Authority Name']==authority]['School Name'].unique():
    print(school)

Since PATs are written in Grades 6 and 9, some schools will have one or both of these grades.

The course name usually ends with a number, for example `Science 6`, so let's find the last character of each course name. We may also see `e` as in `French Language Arts 6 année` or `)` as in `Science 6 (Fr)`, but this should be able to tell us which grades wrote tests in each school.

In [ ]:
for school in data[data['Authority Name']==authority]['School Name'].unique():
    courses = data[data['School Name']==school]['Course Name'].unique()
    grades = []
    for course in courses:
        grade = course[-1:]
        if grade not in grades:
            grades.append(grade)
    print(school, grades)

## Selecting a School

Let's select an elementary school so that we only have one grade represented.

In [ ]:
school = 'Pine Street School'
sd = data[data['School Name']==school]
sd

## Choosing Data

We can start filtering these data by `Language`. The options are `Written in English`, `Written in French`, or `All Students Writing`.

In [ ]:
language = 'Written in English'
sd1 = sd[sd['Language']==language]
sd1

We now have only a few rows, but there are still a lot of columns.

In [ ]:
len(sd1.columns)

### Years

Many of the columns start with a year, so we can see which years are represented in the data.

In [ ]:
years = []
for column in sd1.columns:
    if column[0] == '2':
        year = column[0:4]
        if year not in years:
            years.append(year)
years

### Types of Results

Let's see what we have measurements for in the first year, `years[0]`.

In [ ]:
results_columns = []
for column in sd1.columns:
    if column[0:4] == years[0]:
        results_columns.append(column[5:])
results_columns

### Choosing a Result

Let's select one of those, the percent of students who achieved the "standard of excellence", and select just those columns for each year.

In [ ]:
result = 'Sch % Exc of Writing'
columns_to_graph = ['Course Name']
for year in years:
    columns_to_graph.append(year + ' ' + result)
sd2 = sd1[columns_to_graph]
sd2.columns = ['Course Name'] + years
sd2

### Charting Results

We can now create an interactive bar chart for these data. 

In [ ]:
years = sd2.columns[1:]
px.bar(sd2, x='Course Name', y=years, barmode='group', title=school+' '+result)

If we prefer a chart for just one course we can filter and transpose (`T`) the data.

In [ ]:
course = 'Science 6'
title = course+' '+school+' '+result
px.bar(sd2[sd2['Course Name']==course].set_index('Course Name').T, title=title).update_layout(showlegend=False)

## Other Results

To chart the data from another school or for another result, we can combine code from previous cells. Once again we will be downloading the data so it may take a minute.

In [ ]:
school = 'Lakeland Ridge School'
result = 'Sch % Acc of Writing'
language = 'Written in English'  # or 'Written in French'

pat_url = 'https://www.alberta.ca/assets/documents/ed-pat-multiyear-sch-list.xlsx'
import requests
import pandas as pd
import plotly.express as px
r = requests.get(pat_url)
data = pd.read_excel(r.content)
years = []
for column in sd.columns:
    if column[0] == '2':
        year = column[0:4]
        if year not in years:
            years.append(year)

sd = data[(data['School Name']==school) & (data['Language']==language)]
columns_to_graph = ['Course Name']
for year in years:
    columns_to_graph.append(year + ' ' + result)
sd1 = sd[columns_to_graph]
sd1.columns = ['Course'] + years
px.bar(sd1, x='Course', y=years, barmode='group', title=school+' '+result)

We can also transpose the data if we prefer a chart with years on the x-axis.

In [ ]:
px.bar(sd1.set_index('Course').T, barmode='group', title=school+' '+result).update_xaxes(title='Year').update_yaxes(title=result)

# Conclusion

In this notebook we downloaded, explored, and created charts from [Alberta Education PAT result data](https://www.alberta.ca/provincial-achievement-tests.aspx).

A similar process could be followed using [Diploma Exam](https://www.alberta.ca/diploma-exams.aspx) results, perhaps from the [Alberta Open Data site](https://open.alberta.ca/opendata/diploma-exam-results-and-participation-rates).

For information about interpreting provincial testing results,  please refer to the [Guidelines for Interpreting Achievement Test Multiyear Reports](https://www.alberta.ca/assets/documents/guidelines-internet-pat-multiyear-2019.pdf).

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)